# Project: Cuisine Classified
Authors: 
* Ryan Chang - rchan123
* Ryan Chandler - rchan129
* Kunal Mittal - kmitt006
* Fiorello Estuar - festu001
* Kiet Lam - klam073

# Description

This project aims to analyze the relationships between various flavors found in different foods and the cuisines that they are prominent in. 

# Reading/Cleaning Data
This section handles any cleaning/pre-processing of data such that analysis can take place.

## Reading

In [315]:
# importing libraries
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import chi2_contingency
import json, os

flavor_dict = {'piquant': [], 'meaty': [], 'bitter': [], 'sweet': [], 'sour': [], 'salty': []}
FLAVORS = np.array(list(flavor_dict.keys()))
CUISINES = np.array(['American','Chinese','Cuban','English','French','German','Greek','Hawaiian','Hungarian','Indian','Italian','Japanese','Mexican','Moroccan','Portuguese','Spanish','Swedish','Thai'])

In [316]:
# Reading in the data
food_list_folder = 'data/lists/'
food_list_jsons = [food_json for food_json in os.listdir(food_list_folder) if food_json.endswith('.json')]

food_df = pd.DataFrame(columns=['recipeName','ingredients','cuisine','flavors'])

for index, js in enumerate(food_list_jsons):
    with open(os.path.join(food_list_folder, js)) as food_list_file:
        food_json = json.load(food_list_file)
        num_items = len(food_json['matches'])
        for idx in range(0,num_items):
            f_obj = food_json['matches'][idx]
            f_id = f_obj['id']
            f_recipeName = f_obj['recipeName']
            f_ingredients = f_obj['ingredients']
            f_cuisine = f_obj['attributes']['cuisine']
            f_flavors = f_obj['flavors']
            food_df.loc[f_id] =  np.array([f_recipeName,f_ingredients,f_cuisine,f_flavors],dtype=object)

## Cleaning

In [317]:
# dropping all rows that do not have labels
food_df = food_df[food_df['flavors'].notna()]
food_df.head()

,recipeName,ingredients,cuisine,flavors
Easy-Seafood-Paella-2077829,Easy Seafood Paella,"[lobster tails, water, olive oil, yellow onion...",[Spanish],"{'piquant': 0.6666666666666666, 'meaty': 0.166..."
Spanish-style-garlic-shrimp-298317,Spanish-style Garlic Shrimp,"[I Can't Believe It's Not Butter!® Spread, unc...",[Spanish],"{'piquant': 0.8333333333333334, 'meaty': 0.833..."
Classic-Paella-898818,Classic Paella,"[extra-virgin olive oil, skinless chicken brea...",[Spanish],"{'piquant': 0.16666666666666666, 'meaty': 0.16..."
One-Pot-Spanish-Chicken-and-Rice-2237989,One Pot Spanish Chicken and Rice,"[olive oil, chicken thighs, salt, pepper, onio...",[Spanish],"{'piquant': 0.16666666666666666, 'meaty': 0.16..."
Portuguese-fish-stew-306395,Portuguese Fish Stew,"[plum tomatoes, savoy cabbage, extra-virgin ol...","[Spanish, Portuguese]","{'piquant': 0.5, 'meaty': 0.16666666666666666,..."


In [318]:
# clean the flavors
flavor_dict = {'piquant': [], 'meaty': [], 'bitter': [], 'sweet': [], 'sour': [], 'salty': []}
for flavors in food_df.flavors:    
    for specificFlavor in flavors:
        flavor_dict[specificFlavor].append(flavors[specificFlavor])

for flavorKey in flavor_dict:
    try:
        food_df.insert(len(food_df.columns), flavorKey, flavor_dict[flavorKey])
    except:
        pass

# food_df.drop(columns = ["flavors"], inplace=True)

In [319]:
# clean the cuisines, give each it's own column and label 0 or 1 based on presence
for cus in CUISINES:
    for key, cus_list in food_df.iterrows():
        food_df.loc[key,cus] = 1 if cus in cus_list['cuisine'] else 0
food_df.head()

,recipeName,ingredients,cuisine,flavors,piquant,meaty,bitter,sweet,sour,salty,...,Hungarian,Indian,Italian,Japanese,Mexican,Moroccan,Portuguese,Spanish,Swedish,Thai
Easy-Seafood-Paella-2077829,Easy Seafood Paella,"[lobster tails, water, olive oil, yellow onion...",[Spanish],"{'piquant': 0.6666666666666666, 'meaty': 0.166...",0.666667,0.166667,0.333333,0.166667,0.166667,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Spanish-style-garlic-shrimp-298317,Spanish-style Garlic Shrimp,"[I Can't Believe It's Not Butter!® Spread, unc...",[Spanish],"{'piquant': 0.8333333333333334, 'meaty': 0.833...",0.833333,0.833333,0.833333,0.166667,0.666667,0.833333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Classic-Paella-898818,Classic Paella,"[extra-virgin olive oil, skinless chicken brea...",[Spanish],"{'piquant': 0.16666666666666666, 'meaty': 0.16...",0.166667,0.166667,0.333333,0.166667,0.666667,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
One-Pot-Spanish-Chicken-and-Rice-2237989,One Pot Spanish Chicken and Rice,"[olive oil, chicken thighs, salt, pepper, onio...",[Spanish],"{'piquant': 0.16666666666666666, 'meaty': 0.16...",0.166667,0.166667,0.166667,0.166667,0.833333,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Portuguese-fish-stew-306395,Portuguese Fish Stew,"[plum tomatoes, savoy cabbage, extra-virgin ol...","[Spanish, Portuguese]","{'piquant': 0.5, 'meaty': 0.16666666666666666,...",0.500000,0.166667,0.166667,0.166667,0.500000,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [320]:
# cleaning cuisine list
food_df["cuisine"] = food_df.apply(lambda row: np.intersect1d(np.asarray(row["cuisine"]), CUISINES), axis=1)

In [321]:
#clean the ingredients
ingredientDict = {}
for ingredeintList in food_df.ingredients :
    for ingredient in ingredeintList :
        if ingredient not in ingredientDict :
            ingredientDict[ingredient] = {}

ingredientdf = pd.DataFrame.from_dict(ingredientDict, orient='index')

for index, row in food_df.iterrows() :
    for ingredient in row.ingredients :
        for flavor in row.flavors :
                if(flavor not in ingredientDict[ingredient]) :
                    ingredientDict[ingredient][flavor] = [row.flavors[flavor]]
                else :
                    ingredientDict[ingredient][flavor].append(row.flavors[flavor])

discardList = []
ingredient_freq = []
for ingredient in ingredientDict :
    if len(ingredientDict[ingredient]['piquant']) < 10 :
        discardList.append(ingredient)
        continue
    ingredient_freq.append(len(ingredientDict[ingredient]['piquant']))
    for flavor in FLAVORS :
        ingredientDict[ingredient][flavor] = sum(ingredientDict[ingredient][flavor])/len(ingredientDict[ingredient][flavor])

#print(ingredientDict)
for ingredient in discardList :
    ingredientDict.pop(ingredient)
ingredient_df = pd.DataFrame.from_dict(ingredientDict, orient='index')

ingredient_df['Frequency'] = ingredient_freq

mainFlavorList = []
ingredientList = []
for index, row in ingredient_df.iterrows() :
    flavorList = [row.piquant, row.meaty, row.bitter, row.sweet, row.sour, row.salty]
    mainFlavorList.append(FLAVORS[flavorList.index(max(flavorList))])
    ingredientList.append(index)

ingredient_df["Main Flavor"] = mainFlavorList
ingredient_df["Ingredients"] = ingredientList

ingredient_df.head()


,piquant,meaty,bitter,sweet,sour,salty,Frequency,Main Flavor,Ingredients
water,0.251602,0.313866,0.363069,0.212502,0.405454,0.464843,2029,salty,water
olive oil,0.262075,0.361599,0.357929,0.198106,0.486413,0.428841,3361,sour,olive oil
yellow onion,0.302704,0.319814,0.331643,0.192438,0.471483,0.429024,789,sour,yellow onion
garlic cloves,0.290848,0.351786,0.375818,0.207738,0.492336,0.441964,2240,sour,garlic cloves
saffron threads,0.339976,0.289251,0.265097,0.189614,0.515097,0.366546,276,sour,saffron threads


# Elementary Data Analysis

In [322]:
# cuisines
#   v1. which are the top n cuisines (frequency) in the dataset
#   v2. which cuisines have higher amounts of fusion.
#   v3. which cuisines are most blended with other cuisines (heat map?)

# flavor
#   v1. MMM - using every recipe together at first (broken down by cuisine)
#       - flavors: broken down into the 6 categories
#   v2. for each of the cuisines, visualize the flavors - spider graph
#   v3. why is hawaiian food so sweet?

# ingredients (kiet/ryan)
#   v1. MMM
#   v2. 
#   v3. 


## Cuisine

In [323]:
### Fio's part

### Top cuisines (by frequency) in the dataset
# We create a dictionary where the keys are the names of the cuisines, and the values are the number of recipes which possess that cuisine tag.
dict_cuisine_freq = {}
for cus in CUISINES:
    dict_cuisine_freq[cus] = int(food_df[cus].sum())

# Because each recipe may have more than one cuisine tag, we cannot calculate a proportion off the total number of recipes alone because our numerator
#  would be bigger than the denominator, so we instead calculate our proportions off the total number of tags. This provides the secondary benefit of
#  having non-fusion recipes "count" for more, as they contribute less tags to the total count of recipes.
#  EX: A fusion recipe with 5 (American, Chinese, Hawaiian, Japanese, Indian) tags would only be 1/5 as meaningful to the overall proportion of chinese recipes
#        compared to a recipe with only 1 tag, Chinese.
num_tags = sum(dict_cuisine_freq.values())
# print(dict_cuisine_freq) 
# print(num_tags)


# Calculate the proportion of each cuisine across all recipes, then convert into a dataframe.
dict_proportion = {}
for cus in CUISINES:
    dict_proportion[cus] = dict_cuisine_freq[cus] / num_tags
df_proportions = pd.DataFrame(list(dict_proportion.items()), columns = ['Cuisine', 'Proportion'])

# Sort the dataframe by descending proportion
df_proportions.sort_values(by=['Proportion'], ascending=False)

,Cuisine,Proportion
5,German,0.070454
1,Chinese,0.067600
15,Spanish,0.066262
6,Greek,0.066084
13,Moroccan,0.065549
4,French,0.063230
3,English,0.061090
0,American,0.060644
10,Italian,0.060109
12,Mexican,0.059930


In [324]:
# TODO: Visualization for Cuisine proportion

TODO: Written Analysis for proportion of cuisines 

##  Flavor

In this section, we aim to examine flavors of a given recipe, and any patterns that may exist within each flavor, or it's predicitve power over the various cuisines.

In [325]:
# flavors, and flavor averages
flavor_fig = go.Figure()
for flav in FLAVORS:
    flavor_fig.add_trace(go.Violin(
        y=food_df[flav],
        name=flav,
    ))
flavor_fig.update_layout(
    title_text="Flavors, Visualized"
)
flavor_fig.show()

This visualization shows the distribution of flavors in the various dishes. From this plot, we can see that the flavors were rated on a scale from 1-7, as demonstrated by the spikes in the violin chart. The data tends to be skewed towards the ends; recipes tended to either have an abundance of a flavor, or almost entirely lack it. Consequently, there is significantly less data towards the centers of each flavor spectrum.

Deeper analysis of the flavors in the contexts of different independent variables may allow us to unveil hidden patterns that exist among the various flavor categories.

### Cuisines vs. Flavors

In [326]:
# manually creating pivot table
cuisine_flavor = pd.DataFrame(columns=FLAVORS,index=CUISINES)
for cus in CUISINES:
    for f in FLAVORS:
        cuisine_flavor.loc[cus,f] = food_df.loc[food_df[cus]==1,f].mean()
cuisine_flavor['magnitude'] = cuisine_flavor[FLAVORS].sum(axis=1)/6

In [327]:
# cusines vs. flavors
region_grouping = [
    ["Hawaiian","American","Mexican","Cuban"],
    ["Italian","Portuguese","Spanish","Moroccan"],
    ["English","French","German","Hungarian","Swedish"],
    ["Indian","Thai","Chinese","Japanese"]
]
for group in region_grouping:
    group = sorted(group,key=lambda x: cuisine_flavor.loc[x,"magnitude"])
specs_list = []
for i in range (0,1):
    specs_list.append([])
    for j in range(0,2):
        specs_list[i].append({"type":"polar"})
fig1t = make_subplots(rows=1,cols=2,specs=specs_list,subplot_titles=('Americas','Southern Europe'))
group_num = 0
for i in range(0,2):
    for cuisine in region_grouping[group_num]:
        fig1t.append_trace(
            go.Scatterpolar(
                r=cuisine_flavor.loc[cuisine],
                theta=FLAVORS,
                fill='toself',
                name=cuisine
            ),row=1,col=i+1
        )
    group_num += 1
fig1b = make_subplots(rows=1,cols=2,specs=specs_list,subplot_titles=('Northern Europe','Asia'))
for i in range(0,2):
    for cuisine in region_grouping[group_num]:
        fig1b.append_trace(
            go.Scatterpolar(
                r=cuisine_flavor.loc[cuisine],
                theta=FLAVORS,
                fill='toself',
                name=cuisine
            ),row=1,col=i+1
        )
    group_num += 1
fig1t.update_layout(
    title_text="Cuisine vs. Flavor",
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0,0.75]
        )
    ),
)
fig1b.update_layout(
    title_text="Cuisine vs. Flavor",
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0,0.75]
        )
    ),
)
fig1t.show()
fig1b.show()

TypeError: 'dict' object is not callable

To visualize the flavors present in these various cuisines, we use a spider graph to compare the varying flavor qualities between cuisines. Since there were 18 regions defined in this dataset, it would prove inefficient to overlay all 18 spider charts over eachother. Instead, we proceed by creating separate subgraphs for 4 geographical regions: Americas, Southern Europe, Northern Europe, and Asia. 

Within the cuisines geographically close to the Americas, we can see that there is a high variety of flavors in this region. It is interesting to note that Mexican cuisine is significantly more 'piquant' or 'spicy' than it's neighbouring cuisines. Hawaiian cuisine boasts having the sweetest recipes, with the second highest belonging to Chinese cuisine.

In the Asian cuisines, it is interesting to note that on average, the Asian recipies tended to be significantly more salty and bitter than in other cuisines. Furthermore, Japanese and Chinese cuisines appear to encompass very similar flavors, having similar values in all flavor qualities. 

On the contrary, we can see that cuisines belonging to the Northern European regions did not introduce flavors that were as vibrant as Asian recipes. Unlike in the Americas (where flavors were varied), Northen European flavors tended to stay more-or-less similar between cuisines.

Finally, in Southern European Cuisines, the flavor charts tended to depict similar flavor structures, with high sour levels, but lower salt levels. However, we can see that Italian cuisine is very different from typical Southern European cuisine; rather it resembles a flavor structure similar to Japanese or Chinese cuisine boasting high levels of salty and bitter flavors, and lower levels of piquant and sour flavors. Italian flavors being similar to Asian flavors could be a result of the exchange of ideas/ingredients which occured across the Silk Road.

### Region vs. Flavor

In [ ]:
# 3D Scatter Plot, Recipe vs. Cuisine
# TODO: fix legend labels (has to do with order you add countries to the trace), add title
fig2sub = make_subplots(rows=1,cols=2,specs=[[{'type':'scene'},{'type':'scene'}]])
fig2left = go.Figure(data=[
    go.Scatter3d(
        x=cuisine_flavor.loc[region_grouping[0]]['sweet'],
        y=cuisine_flavor.loc[region_grouping[0]]['sour'],
        z=cuisine_flavor.loc[region_grouping[0]]['salty'],
        text=cuisine_flavor.loc[region_grouping[0]].index,
        mode='markers',
        name='Americas',
        legendgroup='g1',
        marker=dict(
            size=10,
            color="#F51663"
        )
    ),
    go.Scatter3d(
        x=cuisine_flavor.loc[region_grouping[1]]['sweet'],
        y=cuisine_flavor.loc[region_grouping[1]]['sour'],
        z=cuisine_flavor.loc[region_grouping[1]]['salty'],
        text=cuisine_flavor.loc[region_grouping[1]].index,
        mode='markers',
        name='Southern Europe',
        legendgroup='g2',
        marker=dict(
            size=10,
            color="#DEB3AD"
        )   
    ),
    go.Scatter3d(
        x=cuisine_flavor.loc[region_grouping[2]]['sweet'],
        y=cuisine_flavor.loc[region_grouping[2]]['sour'],
        z=cuisine_flavor.loc[region_grouping[2]]['salty'],
        text=cuisine_flavor.loc[region_grouping[2]].index,
        mode='markers',
        name='Northern Europe',
        legendgroup='g3',
        marker=dict(
            size=10,
            color="#FAC218"
        )
    ),
    go.Scatter3d(
        x=cuisine_flavor.loc[region_grouping[3]]['sweet'],
        y=cuisine_flavor.loc[region_grouping[3]]['sour'],
        z=cuisine_flavor.loc[region_grouping[3]]['salty'],
        text=cuisine_flavor.loc[region_grouping[3]].index,
        mode='markers',
        name='Asia',
        legendgroup='g4',
        marker=dict(
            size=10,
            color="#F86F15"
        )
    )]
)
fig2right = go.Figure(data=[
    go.Scatter3d(
        x=cuisine_flavor.loc[region_grouping[0]]['piquant'],
        y=cuisine_flavor.loc[region_grouping[0]]['bitter'],
        z=cuisine_flavor.loc[region_grouping[0]]['meaty'],
        text=cuisine_flavor.loc[region_grouping[0]].index,
        mode='markers',
        name='Americas',
        legendgroup='g1',
        showlegend=False,
        marker=dict(
            size=10,
            color="#F51663"
        )
    ),
    go.Scatter3d(
        x=cuisine_flavor.loc[region_grouping[1]]['piquant'],
        y=cuisine_flavor.loc[region_grouping[1]]['bitter'],
        z=cuisine_flavor.loc[region_grouping[1]]['meaty'],
        text=cuisine_flavor.loc[region_grouping[1]].index,
        mode='markers',
        name='Southern Europe',
        legendgroup='g2',
        showlegend=False,
        marker=dict(
            size=10,
            color="#DEB3AD"
        )   
    ),
    go.Scatter3d(
        x=cuisine_flavor.loc[region_grouping[2]]['piquant'],
        y=cuisine_flavor.loc[region_grouping[2]]['bitter'],
        z=cuisine_flavor.loc[region_grouping[2]]['meaty'],
        text=cuisine_flavor.loc[region_grouping[2]].index,
        mode='markers',
        name='Northern Europe',
        legendgroup='g3',
        showlegend=False,
        marker=dict(
            size=10,
            color="#FAC218"
        )
    ),
    go.Scatter3d(
        x=cuisine_flavor.loc[region_grouping[3]]['piquant'],
        y=cuisine_flavor.loc[region_grouping[3]]['bitter'],
        z=cuisine_flavor.loc[region_grouping[3]]['meaty'],
        text=cuisine_flavor.loc[region_grouping[3]].index,
        mode='markers',
        name='Asia',
        legendgroup='g4',
        showlegend=False,
        marker=dict(
            size=10,
            color="#F86F15"
        )
    )]
)
for d in fig2left.data:
    fig2sub.append_trace(d,row=1,col=1)
for d in fig2right.data:
    fig2sub.append_trace(d,row=1,col=2)
fig2sub.update_scenes(
    xaxis=dict(title_text='piquant'),
    yaxis=dict(title_text='bitter'),
    zaxis=dict(title_text='meaty')
)
fig2sub.update_layout(
    title_text="Cuisine Region vs. Flavor",
    scene=dict(
    xaxis_title='sweet',
    yaxis_title='sour',
    zaxis_title='salty'
))
fig2sub.show()

We can further explore the flavor trends between various geographical regions using 2 3-Dimensional scatterplots. With these scatterplots, we can verify that in general cuisines that are regionally closer together tended to share similar flavor characteristics. These 3d scatter plots allow us to more clearly compare the differences in flavors between the cuisines when clustered by their geographical regions.

As we saw in the radar charts, all cuisines had lower sweet levels, with the exception of Hawaiian cuisine. 

Various patterns are revealed when comparing the cuisines using the 3D scatterplot. From the first subplot, with the exception of Italian cuisine, Southern European flavors tended to more sour, and minimally salty and sweet. On the contrary, American and Asian flavors tended to be more vibrant, boasting higher levels of saltiness and sourness.

These patterns remain consistent with the second subgraph. European flavors on average were less piquant and bitter than Asian or American cuisines. There is little variance between all cuisines with respect to meaty flavors. 

From these visualizations, we observe that objectively, European cuisines tended to be less vibrant in flavors when compared to Asian cuisine. The more vibrant flavors present in Asian cuisines could be a result to more herbs/spices being used in Asian cuisines, when compared to European cuisines. 

### Why is Hawaiian Cuisine so sweet?

As we saw in the prior two visualizations, Hawaiian Cuisine tends to be much sweeter than every other cuisine. In this section, we attempt to justify this observation.

In [ ]:
# get top ingredients k that are part of hawaiian ingredients
k = 7
top_hawaiian_ingredients = {}
for id, f_row in food_df.iterrows():
    if f_row.Hawaiian == 1:
        for ingredient in f_row.ingredients:
            if ingredient not in top_hawaiian_ingredients:
                top_hawaiian_ingredients[ingredient] = 1
            else:
                top_hawaiian_ingredients[ingredient] += 1
top_hawaiian_ingredients = list(dict(sorted(top_hawaiian_ingredients.items(), key=lambda item: item[1],reverse=True)[:k]).keys())

In [ ]:
# find occurences of top k ingredients in all other cuisines
hawaiian_ingredient_in_cuisines = pd.DataFrame(columns=top_hawaiian_ingredients,index=CUISINES)
hawaiian_ingredient_in_cuisines.iloc[:] = 0
for id, f_row in food_df.iterrows():
    for hawaiian_ingredient in top_hawaiian_ingredients:
        if hawaiian_ingredient in f_row.ingredients: 
            for cur_cus in CUISINES:
                if f_row[cur_cus] == 1:
                    hawaiian_ingredient_in_cuisines.loc[cur_cus,hawaiian_ingredient] += 1
for cus in dict_cuisine_freq:
    hawaiian_ingredient_in_cuisines.loc[cus] =  hawaiian_ingredient_in_cuisines.loc[cus] / dict_cuisine_freq[cus]
hawaiian_ingredient_in_other_cuisines = hawaiian_ingredient_in_cuisines.loc[np.delete(CUISINES,np.where(CUISINES=='Hawaiian'))]
fig4 = go.Figure(data=[
    go.Bar(name='Hawaiian', 
        x=hawaiian_ingredient_in_cuisines.loc['Hawaiian'].keys(), 
        y=hawaiian_ingredient_in_cuisines.loc['Hawaiian'],
        marker=dict(
            color='#E3B448',
        )
    ),
    go.Bar(name='Avg. of other Cuisines', 
        x=hawaiian_ingredient_in_other_cuisines.keys(), 
        y=hawaiian_ingredient_in_other_cuisines.mean(axis=0),
        marker=dict(
            color='#3A6B35',
        )
    )
])
fig4.update_layout(
    title_text="Proportion of Top Hawaiian Ingredients in Hawaiian Cuisines vs. Other Cuisines"
)
fig4.show()

As we saw in the spider chart and the 3d scatterplot, Hawaiian cuisine bragged exceptionally high levels of sweet flavors in their cuisine. By analyzing the ingredient composition of Hawaiian dishes, we can provide reasoning to high levels of sweetness.

We first gather the most frequent ingredients used in Hawaiian cuisine. These ingredients are: soy sauce, salt, brown sugar, pineapple/pineapple chunks, pineapple juice, and corn starch. We then compare the proportion of each ingredient used in each recipe, by cuisine.

Pineapples, and pineapple byproducts are used at an outstandingly high rate (approx. 30% of recipies) in Hawaiian recipes in comparison to the rest of the world's cusines (approx. 0.2% of recipies). This could be a result of the abundance of pineapples present in Hawaii, when compared to the presense of pineapples in other regions of the world. Pineapples, being a sweet ingredient, could explain why Hawaiian dishes tended to be more sweet than other cuisines' dishes.

Furthermore, brown sugar is used at an extraordinarily high rate in Hawaiian cuisines (approx. 32%), when compared to the average presence in other cuisines (approx. 5%). Similarly to pineapples, brown sugar is a sweet ingredient, and thus dishes with brown sugar present in them are more sweet.

Of the top 7 most frequently used ingredients in Hawaiian cuisine, 4 of them are sweet ingredients, and these ingredients are used, on average, at an absurdly high rate compared to the usage in the average cuisine. Thus, we observe that the ingredient composition of Hawaiian ingredients are a contributing factor to the sweet flavors of Hawaiian dishes.

## Ingredients and Flavor

### Which ingredients have the most flavor?

We want to know which ingredients play the most/least ingredients role in determining a recipes flavor.

In [ ]:
dict = {'Flavor': [], 'Ingredient' : [], 'Flavor Value' : [], 'How Flavorful?' : [], 'Frequency' : []}
plot_df = pd.DataFrame(dict)
for flavor in FLAVORS :
    plot_df.loc[len(plot_df.index)] = [flavor, ingredient_df.loc[:, flavor].idxmax(), ingredient_df.loc[:, flavor].max(), 'Most Flavorful', ingredient_df.loc[(ingredient_df.loc[:, flavor].idxmax()), 'Frequency']]
for flavor in FLAVORS :
    plot_df.loc[len(plot_df.index)] = [flavor, ingredient_df.loc[:, flavor].idxmin(), ingredient_df.loc[:, flavor].min(), 'Least Flavorful', ingredient_df.loc[(ingredient_df.loc[:, flavor].idxmin()), 'Frequency']]
    
fig11 = px.scatter(plot_df, x = 'Flavor', y = 'Flavor Value', color = 'How Flavorful?', hover_name="Ingredient", size = "Frequency", size_max=60, title = "Most and Least Flavorful Ingredients")
fig11.show()


In order to understand how ingredients affect the flavor of a particular recipe, we used a bubble plot that illustrates the most and least flavorful ingredients of each of the six flavors. For recipes that use these "Most Flavorful" ingredients, we can assume that their flavors will be focused around the flavor of their ingredients. For example, recipes that use the "yellow pepper" are more likely to be sour we categorized yellow pepper as a categorized as a "sour" ingredient and has the strongest flavor among "sour" ingredients.

Conversely, recipes that make use of "Least Flavorful" ingredients are less likely to have their flavor influenced by these ingredients. This is useful, because we can use this data to explain why certain geographical cuisines have developed a certain type of flavor.

For example, as shown in a previous chart, Mexican cusine is the most "Piquant" cuisine out of all. As seen from the "Most and Least Flavorful Ingredients" chart, the most piquant ingredient is worceteshire sauce. As it turns out, worceteshire sauce is a staple in Mexican Cusine, and it also is known by its other name, salsa inglesa. Therefore, we can assume that Mexican cuisine obtains a lot of it's piquant flavor from salsa inglesa.

However it's important to remember that correlation does not equal causation. For example, "hot dog buns" are the most flavorful ingredient within the "salty" category. On their own, "hot dog buns" aren't particularly salty, but they are often combined and used with other salty dishes, thus making it a "salty" ingredient.

### How does an ingredient's flavor interact with other flavors?

Because ingredients have multiple flavor values, we want to observe how they compare amongst each other.

In [ ]:
fig14 = px.scatter_ternary(ingredient_df, a="piquant", b="meaty", c="bitter",
    color= 'Main Flavor', size="Frequency", size_max=60, hover_name='Ingredients', title = "Piquantness, Bitterness, and Meatyness of Each Ingredient")
fig14.show()

In [ ]:
fig15 = px.scatter_ternary(ingredient_df, a="sweet", b="sour", c="salty",
    color= 'Main Flavor', size="Frequency", size_max=60, hover_name='Ingredients', title = "Sweetness, Sourness, and Saltyness of Each Ingredient")
fig15.show()

Similar to recipes, ingredients often have numerous flavor values. We can manipulate the data as much as we want in order to obtain an ingredients generalized "Main Flavor" or an average "Flavor Value", but there's still several underlying flavors within a particular ingredient. Using a ternary chart, we can see how these various flavors interact with each of the ingredients in our data frame. For example, according to the first chart, "salty" ingredients seem to also have really high "bitter" values compared to other ingredients.

Because the ternary chart only can compare three flavors at once, we decided to make two charts to comparing three flavors to obtain visualization data for all six of our flavors. We decied to group the flavors based on proximity to each other, so the first illustrates the "Piquantness, Bitterness, and Meatyness of Each Ingredient", and the second chart is the "Sweetness, Sourness, and Saltyness of Each Ingredient".

This is a useful form of data representation because we can hover over an interesting dat point and see which ingredient it is. Additionally, the coloring being based upon our "Main Flavor" metric can reveal the accuracy (or even inaccuracies) of this form of classification. For example, from the first chart, we can see that the ingredients with "Main Flavor" of "piquant" and "meaty" are generally in their respective corners towards the top, and bottom-left. However, "bitter" flavor values appear to be more focused towards the center than the other two flavors. Even though the tenary chart is only comparing ingredients to three flavors at a time, the "Main Flavor" category allows us to observe patterns regarding the other flavors that aren't as directly represented within the chart.

It's worth nothing that in the second chart, "Sweetness, Sourness, and Saltyness of Each Ingredient", there are no virtually no ingredients that contain any high values of sweetness. This makes sense given the previous Bubble Chart, in which even the sweetest ingredient contained a flavor value flavor value was significantly lower than other flavorful ingredients. Therefore, the ingredients on a Ternary Chart not congolmerating towards the sweet corner makes a lot of sense given the low flavor values of the "sweet" flavor.


# Statistical Analysis

# K-Nearest Neighbors Classifier

In [ ]:
#ML STUFF
# [0.15,0.1,0.05,0.4,0.35,0.65]
# {"piquant": 0.6666666666666666, "meaty": 0.3333333333333333, "bitter": 0.8333333333333334, "sweet": 0.6666666666666666, "sour": 0.16666666666666666, "salty": 0.8333333333333334}
# [0.6666666666666666,0.3333333333333333,0.8333333333333334,0.6666666666666666,0.16666666666666666,0.8333333333333334]
def multiKNN(end: int):
    neighbordata = food_df.loc[:, FLAVORS].values.tolist()
    def knn(unknown, data, neighbors: int):
        classifier = NearestNeighbors(n_neighbors=neighbors)
        classifier.fit(data)
        return classifier.kneighbors([unknown], return_distance=False)

    countCuisines = []
    for i in range(1, end):
        closestpts = knn([0.6666666666666666,0.3333333333333333,0.8333333333333334,0.6666666666666666,0.16666666666666666,0.8333333333333334], neighbordata, i)
        finalCuisine = dict(zip(CUISINES, [0] * len(CUISINES)))

        for listFoods in closestpts:
            for food in listFoods:
                cuisineList = food_df.iloc[food, food_df.columns.get_loc("cuisine")]
                for cuisine in cuisineList:
                    finalCuisine[cuisine] += 1/len(cuisineList)

        typefood = [key for key, value in finalCuisine.items() if value == max(finalCuisine.values())]
        countCuisines.append(typefood)
    return countCuisines

numFinalNeighbors = 20
total = multiKNN(numFinalNeighbors)
finalfinalCuisine = dict(zip(CUISINES, [0] * len(CUISINES)))

for resultsList in total:
    for cuisine in resultsList:
        finalfinalCuisine[cuisine]+=1

end = [key for key, value in finalfinalCuisine.items() if (value == max(finalfinalCuisine.values()) or abs(value-max(finalfinalCuisine.values()))<=numFinalNeighbors//10)]
end

['American', 'Hawaiian', 'Mexican']

In [ ]:
# chi-square for sweetness vs hawaiin food 
hawaiiantosweet = pd.crosstab(food_df["sweet"], food_df["Hawaiian"], margins=True)
c, p, dof, expctd = chi2_contingency(hawaiiantosweet)
c, p, dof

(2363.807105277691, 0.0, 14)

In [ ]:
indiantopiquant = pd.crosstab(food_df["piquant"], food_df["Indian"])
c, p, dof, expctd = chi2_contingency(indiantopiquant)
c, p, dof

(396.67837970489256, 1.4472735107671964e-82, 6)